Imports que importam :) 

In [1]:
import numpy as np
import math

import pandas

In [2]:
#tirar dependencias das libs:
rand = lambda :np.random.rand()
cutpart = lambda x: (int((x+1) * np.random.normal(scale=0.2,loc=0.5)) )% x

In [3]:
va = [x for x in range(1,11)]
vb = [x for x in range(10,0,-1)]
print("va:\n{}\nvb:\n{}".format(va,vb))

va:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
vb:
[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


<h1>Genetic Algorithm for QAP solving()</h1>

In [4]:
def crossover(va,vb):
    cut = cutpart(len(va))
    print("cut: {}".format(cut))
    new_va = va[:cut]+vb[cut:]
    new_vb = vb[:cut]+va[cut:]
    print("new va:\n{}\nnew vb:\n{}".format(new_va,new_vb))
    #encontrar repetidos (x) e suas repectivas posicoes (i)
    repeated_a = [(x,i) for x,i in zip(new_va,range(len(va))) if new_va.count(x) > 1]
    repeated_b = [(x,i) for x,i in zip(new_vb,range(len(vb))) if new_vb.count(x) > 1]
    rn = int(len(repeated_a)/2)
    print("rn:{}".format(rn))
    for i in range(rn):
        print("switch {} / {}".format(i,rn))
        new_va[repeated_a[i][1]] = repeated_b[i+ (0 if cut >5 else rn)][0]	#se trocar sempre o primeiro 
        new_vb[repeated_b[i][1]] = repeated_a[i+ (0 if cut >5 else rn)][0]	#repetido cortes acima de 5 tendem
    print("repeated a: {}".format(repeated_a[:int(len(repeated_a))]))		#a repertir os valores dos pais
    print("repeated b: {}".format(repeated_b[:int(len(repeated_b))]))
    return(new_va,new_vb)

In [5]:
new_va,new_vb = crossover(va,vb)#[1,5,3,4,6,10,7,9,8,2])
print("new va:\n{}\nnew vb:\n{}".format(new_va,new_vb))

cut: 6
new va:
[1, 2, 3, 4, 5, 6, 4, 3, 2, 1]
new vb:
[10, 9, 8, 7, 6, 5, 7, 8, 9, 10]
rn:4
switch 0 / 4
switch 1 / 4
switch 2 / 4
switch 3 / 4
repeated a: [(1, 0), (2, 1), (3, 2), (4, 3), (4, 6), (3, 7), (2, 8), (1, 9)]
repeated b: [(10, 0), (9, 1), (8, 2), (7, 3), (7, 6), (8, 7), (9, 8), (10, 9)]
new va:
[10, 9, 8, 7, 5, 6, 4, 3, 2, 1]
new vb:
[1, 2, 3, 4, 6, 5, 7, 8, 9, 10]


In [6]:
def inverse(vec):
    inf = int(rand()*len(vec))
    part = list(vec[inf:])
    part.reverse()
    return vec[:inf]+part

In [7]:
def swap (vec):
    n_vec = list(vec)
    p1 = int(rand()*len(vec))
    p2 = int(rand()*len(vec))
    v1 = vec[p1]
    v2 = vec[p2]
    print("swap {} and {}".format(v1,v2))
    n_vec[p1] = v2
    n_vec[p2] = v1
    return n_vec

In [8]:
def insertion(vec):
    inf = int(rand()*len(vec))
    part = vec[inf:]
    part = part[1:] + part[:1]
    print(part)
    return vec[:inf]+part

In [9]:
def mutation(vec,p):#percentual
    if rand() > p:
        return vec
    typ = rand()
    if(typ < 1/3):
        print("inverse")
        vec = inverse(vec)
    elif(typ < 2/3):
        print("swap")
        vec = swap(vec)
    else:
        print("insertion")
        vec = insertion(vec)
    return vec
    
#mutation(va,0.7)
print("Original:")
print(va)
print("Inverse:")
print(inverse(va))
print("Swap:")
print(swap(va))
print("Insertion")
print(insertion(va))

Original:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Inverse:
[1, 2, 3, 4, 5, 10, 9, 8, 7, 6]
Swap:
swap 6 and 1
[6, 2, 3, 4, 5, 1, 7, 8, 9, 10]
Insertion
[6, 7, 8, 9, 10, 5]
[1, 2, 3, 4, 6, 7, 8, 9, 10, 5]


In [10]:
mutation(va,0.7)

insertion
[10, 9]


[1, 2, 3, 4, 5, 6, 7, 8, 10, 9]

<h1>Calculo de Coorelação:</h1>

In [11]:
def corr(x,y,p=False):
    # n
    n = len(x)
#Somatórios:
    #superiores
    xysum = np.dot(x,y) #soma dos x*y
    xsum = sum(x)
    ysum = sum(y)
    #inferiores
    #soma dos quadrados
    x2sum = np.dot(x,x) #multiplica entrada a entrada dos vetores x e ele mesmo e soma
    y2sum = sum([v*v for v in y]) #forma na unha de fazer o mesmo
    #quadrado das somas:
    xsum2 = xsum**2
    ysum2 = ysum**2
    #parte superior:
    sup = (xysum - xsum*ysum/n)
    #parte inferior:
    infe = math.sqrt ( (x2sum - xsum2/n) * (y2sum - ysum2/n) )
    corresult = sup/infe
    if(p):
        print("Taxa de correlação entre X e Y é:")
        print("{}\n".format(sup)+len(str(sup))*"-"+" =  {}".format(corresult)+"\n{}".format(infe))
    return corresult

<h2>Teste Da Correlação</h2>

In [12]:
gerax = lambda base,n,dif: [base + x*dif for x in range(n)] # modo preguiçoso de gerar n números espaçados em 0.5
#Carregar valores(teste):
x = gerax(2.5, 6,0.5)
y = [0.4,0.4,0.3,0.2,0.2,0.1]
corr(x,y,True)

Taxa de correlação entre X e Y é:
-0.5499999999999998
------------------- =  -0.9710083124552239
0.5664215155988811


-0.9710083124552239

In [13]:
#!/usr/bin/python

from pathlib import Path

directory = './nodes_links/'
allnodes = dict()
alllinks = dict()
names = list()
for path in Path(directory).iterdir():
    url = str(path)
    netname = path.name.split('_')[0]
    if(path.name.count('nodes')==1):
        p = allnodes
    else:
        p = alllinks
    try:
        item = pandas.read_csv(url)
        p.update( {netname : item })
        print(url+" lido")
    except:
        try:
            #mudar o encoding para os caracteres
            item = pandas.read_csv(url,encoding="iso-8859-1")
            p.update( {netname : item })
            print(url+" lido")
        except:
            print("!!! não foi possivel ler > "+name+" < !!!")
            pass
    if names.count(netname) == 0:
        names.append(netname) 


nodes_links/canarie_links.csv lido
nodes_links/italy_nodes.csv lido
nodes_links/vbns_nodes.csv lido
nodes_links/loni_links.csv lido
nodes_links/germany_links.csv lido
nodes_links/geant2_links.csv lido
nodes_links/germany_nodes.csv lido
nodes_links/arpanet_links.csv lido
nodes_links/cesnet_links.csv lido
nodes_links/newnet_nodes.csv lido
nodes_links/rnpBrazil_nodes.csv lido
nodes_links/pionier_nodes.csv lido
nodes_links/loni_nodes.csv lido
nodes_links/spain_links.csv lido
nodes_links/renater_nodes.csv lido
nodes_links/lambdaRailUsa_nodes.csv lido
nodes_links/metrona_UK_nodes.csv lido
nodes_links/rnpBrazil_links.csv lido
nodes_links/arnes_links.csv lido
nodes_links/spain_nodes.csv lido
nodes_links/bren_nodes.csv lido
nodes_links/portugal_links.csv lido
nodes_links/arpanet_nodes.csv lido
nodes_links/usaGde_nodes.csv lido
nodes_links/sanet_links.csv lido
nodes_links/nsfnet_nodes.csv lido
nodes_links/pionier_links.csv lido
nodes_links/austria_links.csv lido
nodes_links/nsfnet_links.csv lido

In [14]:
print(names)
netlinks = alllinks['portugal']
netnodes = allnodes['portugal']

['canarie', 'italy', 'vbns', 'loni', 'germany', 'geant2', 'arpanet', 'cesnet', 'newnet', 'rnpBrazil', 'pionier', 'spain', 'renater', 'lambdaRailUsa', 'metrona', 'arnes', 'bren', 'portugal', 'usaGde', 'sanet', 'nsfnet', 'austria', 'viaDatacenterNet', 'coxUsa', 'mzima', 'scteste', 'eon', 'memorexEurope', 'deutschTelecom', 'internet2Usa', 'OmnicomEurope']


In [15]:
#u = pandas.read_csv("./nodes_links/renater_links.csv")
a = netnodes['Id'].values.tolist()
print("nodes: {}".format(a))
print("links: {}".format(netlinks))

nodes: ['Alcácer_do_Sal', 'Aveiro', 'Beja', 'Braga', 'Bragança', 'Caldas_Rainha', 'Castelo_Branco', 'Coimbra', 'Elvas', 'Évora', 'Faro', 'Funchal', 'Guarda', 'Leiria', 'Lisboa', 'Ponta_Delgada', 'Portalegre', 'Portimão', 'Porto', 'Santarém', 'S_João_Madeira', 'Setúbal', 'Sines', 'Viana_do_Castelo', 'Vila_Real', 'Viseu']
links:                 From              To  Length  Capacity    Cost
0             Lisboa   Ponta_Delgada    1500         1  975000
1             Lisboa         Funchal    1050         1  682500
2      Ponta_Delgada         Funchal    1050         1  682500
3           Portimão         Funchal    1010         1  656500
4              Braga        Bragança     209         1  135850
5           Santarém      Portalegre     144         1   93600
6               Beja            Faro     140         1   91000
7           Portimão           Sines     139         1   90350
8           Santarém         Coimbra     136         1   88400
9           Bragança       Vila_Real     

In [16]:
netlinks.loc[:, ['From','To']]
#import networkx as nx
#g = nx.DiGraph()
#nodes = [key for key in u.groupby("From").indices]
nodes = netnodes['Id'].values.tolist()
print(nodes)

matrix = np.matrix(np.zeros( (len(nodes),len(nodes)),dtype=np.int ) )
for n in nodes:
    l = netlinks.loc[netlinks['From']==n,['To']].values
    indexes = [nodes.index(x) for x in l]
    if(len(indexes) > 0):
        matrix[nodes.index(n),indexes] = 1
        matrix[indexes,nodes.index(n)] = 1 #bidirecional
print(matrix)
#l = u.loc[u['From']=="Boston",['To']]
#print(l.values)
#print(nodes.index("NewYork"))
#nodes.index("Boston")
#g.add_nodes_from(nodes)

#[(x,y) for x in ]

['Alcácer_do_Sal', 'Aveiro', 'Beja', 'Braga', 'Bragança', 'Caldas_Rainha', 'Castelo_Branco', 'Coimbra', 'Elvas', 'Évora', 'Faro', 'Funchal', 'Guarda', 'Leiria', 'Lisboa', 'Ponta_Delgada', 'Portalegre', 'Portimão', 'Porto', 'Santarém', 'S_João_Madeira', 'Setúbal', 'Sines', 'Viana_do_Castelo', 'Vila_Real', 'Viseu']
[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0

In [51]:
def calc_grau(matrix):
    soma=0
    print(len(matrix))
    for l in range(len(matrix)-1):
        for c in range(len(matrix)-1):
            soma += matrix[l,c]
    return soma/len(matrix)

In [54]:
print("grau médio: {}".format(calc_grau(matrix)))

26
grau médio: 2.5384615384615383


In [ ]:
def aumentar_grau():
    pass